In [4]:
import pandas as pd
import json

# 데이터 불러오기 및 전처리
df = pd.read_csv('data/교통사고 데이터/전국교통사고다발지역표준데이터.csv', encoding='euc-kr')
drop_list = ['데이터기준일자', '제공기관코드', '제공기관명']
df = df.drop(columns=drop_list)

# '위치코드'의 앞 5자리 숫자로 그룹화하고 사고건수를 합산
df['위치코드_시군구'] = df['위치코드'].astype(str).str[:5]
df_grouped = df.groupby('위치코드_시군구', as_index=False)['사고건수'].sum()
df_grouped = df_grouped.dropna(subset=['위치코드_시군구'])
df_grouped = df_grouped['사고건수'].fillna(0)


# 결과를 JSON 파일로 저장
df_grouped.to_json('data/accident_data.json', orient='records', force_ascii=False)

In [1]:
# 다른 표 데이터를 JSON으로 저장할 수도 있음 (원하는 경우)
df_sorted = df.sort_values(by='사고건수', ascending=False)
df_sorted[['사고지역위치명', '사고건수']].head(20).to_json('data/top20_accident_locations.json', orient='records', force_ascii=False)

# 법정구역 GeoJSON 파일에서 시군구명과 위치코드 매핑 정보 불러오기
with open('data/법정구역_시군구.geojson', 'r', encoding='utf-8') as f:
    geojson = json.load(f)

# GeoJSON에서 시군구명과 위치코드 추출
location_mapping = {}
for feature in geojson['features']:
    code = feature['properties']['SIG_CD']
    name = feature['properties']['SIG_KOR_NM']
    location_mapping[code] = name

# df_grouped에 시군구명 추가
df_grouped['시군구명'] = df_grouped['위치코드_시군구'].map(location_mapping)

# 시군구명 기준으로 사고건수 정렬
df_grouped_sorted = df_grouped.sort_values(by='사고건수', ascending=False)

# 시군구명이 null인 행을 제외하고 상위 20개 데이터 선택
df_grouped_sorted = df_grouped_sorted.dropna(subset=['시군구명']).head(20)

# 결과를 JSON 파일로 저장 (시군구명 포함)
df_grouped_sorted[['시군구명', '사고건수']].to_json('data/top20_accident_grouped.json', orient='records', force_ascii=False)
